# Parameter drift : Indentifiability

# Purpose
If the matematical model is not correct or too little data is available this may lead to paramter drift, so that the parameters in the matematical model changes depending on how the fitted data has been sampled.

# Methodology
* Sample data of forces from a higher order model
* Fit a lower order model to a random sample of this data

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models.regression import ForceRegression, results_summary_to_dataframe
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
from vessel_manoeuvring_models.visualization.regression import show_pred, show_pred_captive
from vessel_manoeuvring_models.visualization.plot import track_plot,captive_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm 
import vessel_manoeuvring_models.models.vmm_martin as vmm_simpler

from vessel_manoeuvring_models.models.vmm import ModelSimulator

from vessel_manoeuvring_models.data.wpcc import ship_parameters, df_parameters, ps, ship_parameters_prime, ps_ship, scale_factor


In [ ]:
#format the book
import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()

## Load model

In [ ]:
model = ModelSimulator.load('../models/model_VCT_abkowitz.pkl')

In [ ]:
u0_=2
angle_deg = 20
result = model.zigzag(u0=u0_, angle=angle_deg)

In [ ]:
result.track_plot();
result.plot(compare=False);

In [ ]:
df_result = result.result.copy()
df_result_prime = model.prime_system.prime(df_result, U=df_result['U'])

In [ ]:
def variate(df, variation_keys, N=10):
    
    variations = []
    for variation_key in variation_keys:
        variation = np.linspace(df[variation_key].min(),df[variation_key].max(), N)
        variations.append(variation)
    
    matrix = np.meshgrid(*variations)
    df_variation = pd.DataFrame()
    for variation_key,values in zip(variation_keys,matrix):
        df_variation[variation_key] = values.flatten()
    
    return df_variation

In [ ]:
variations = {
    'Rudder angle' : ['delta'],
    'Drift angle' : ['v'],
    'Circle' : ['r'],
    'resistance' : ['u'],
    "Rudder and drift angle" : ['delta','v'],
    "Circle + Drift" : ['r','v'],
    
}

N = 10
V_ = u0_

inputs_base = {}
inputs_base['u'] = df_result_prime['u'].mean()
inputs_base['v'] = 0
inputs_base['r'] = 0
inputs_base['delta'] = 0

df_inputs = pd.DataFrame()
for test_type, variation_keys in variations.items():

    inputs = variate(df=df_result_prime, variation_keys=variation_keys, N=N)
    
    for column in list(set(inputs_base.keys())-set(variation_keys)):
        inputs[column]=inputs_base[column]
    
    inputs['test type'] = test_type
    df_inputs = df_inputs.append(inputs, ignore_index=True)
    
df_outputs = model.forces(df_inputs)
df_captive = pd.concat([df_inputs,df_outputs], axis=1)


In [ ]:
captive_plot(df_captive=df_captive, suffixes=[], 
             legends = ['VCT'], styles=['.', '-'])

## Fit a lower order model to this captive dataset

### Regression

In [ ]:
reg = ForceRegression(vmm=vmm, data=df_captive)
display(reg.show_pred_X())
display(reg.show_pred_Y())
display(reg.show_pred_N())

In [ ]:
parameters = pd.DataFrame()
parameters['prime'] = model.parameters
model_vct = reg.create_model(df_parameters=parameters, ship_parameters=model.ship_parameters, 
                             ps=model.prime_system, control_keys=['delta'])

In [ ]:
outputs = model_vct.forces(inputs = df_inputs)
df_captive_all = pd.merge(left=df_captive, right=outputs, 
                      how='left', 
                      left_index=True, 
                      right_index=True,
                      suffixes = ('','_model'),
                      )

captive_plot(df_captive=df_captive_all, suffixes=['_model'], 
             legends = ['VCT', 'model'], styles=['.', '-'])

In [ ]:
#result_vct = model_vct.zigzag(u0=u0_, angle=angle_deg)
result_vct = model_vct.simulate(df_result.loc[0:6])

In [ ]:
result_vct.track_plot(compare=True);
result_vct.plot(compare=True);

In [ ]:
variation_keys = ['u','v','r','delta']
df_inputs = variate(df=df_result_prime, variation_keys=variation_keys, N=8)
df_outputs = model.forces(df_inputs)
df_captive_all = pd.concat([df_inputs,df_outputs], axis=1)

In [ ]:
len(df_captive_all)

In [ ]:
reg_all = ForceRegression(vmm=model, data=df_captive_all)
display(reg_all.show_pred_X())
display(reg_all.show_pred_Y())
display(reg_all.show_pred_N())

In [ ]:
model_all = reg_all.create_model(df_parameters=parameters, ship_parameters=model.ship_parameters, 
                             ps=model.prime_system, control_keys=['delta'])

In [ ]:
#result_all = model_all.simulate(df_result)
result_all = model.simulate(df_result)

In [ ]:
result_all.plot_compare();

In [ ]:
df_compare_parameters =pd.DataFrame()
df_compare_parameters['model'] = model.parameters
df_compare_parameters['model captive all'] = model_all.parameters
df_compare_parameters['model captive 1'] = model_vct.parameters


df_compare_parameters['model_abs'] = df_compare_parameters['model'].abs()
df_compare_parameters.sort_values(by='model_abs', ascending=False, inplace=True)
df_compare_parameters.drop(columns=['model_abs'], inplace=True)

df_compare_parameters = df_compare_parameters.divide(df_compare_parameters['model'], axis=0)

df_compare_parameters['dof'] = pd.Series(df_compare_parameters.index).apply(lambda x:x[0]).values

for dof, df_ in df_compare_parameters.groupby(by='dof', sort=False):
    fig,ax=plt.subplots()
    fig.set_size_inches(10,2)
    df_.plot(kind='bar', ax=ax)
    fig.suptitle(dof)

In [ ]:
df_captive = df_VCT_prime.copy()
df_captive['test type'] = df_VCT['test type']

N = len(df_captive)
N_sample = N - 5
df_captive_sample = df_captive.sample(n=N_sample, random_state=42)

In [ ]:
N

In [ ]:
model_names = [f'{i}' for i in range(20)]

df_captive_all = df_captive.copy()
np.random.seed(42)

models = {}

for model_name in model_names:
    
    df_captive_sample = df_captive.sample(n=N_sample)
    
    reg = ForceRegression(vmm=vmm, data=df_captive_sample)
    model_reg = reg.create_model(df_parameters=df_parameters, ship_parameters=ship_parameters, ps=ps)
    
    models[model_name] = model_vct = reg.create_model(df_parameters=df_parameters, ship_parameters=ship_parameters, ps=ps)
    
    
    outputs = model_reg.forces(inputs = df_captive)
    df_captive_all = pd.merge(left=df_captive_all, right=outputs, 
                          how='left', 
                          left_index=True, 
                          right_index=True,
                          suffixes = ('',f'_{model_name}'),
                          )

In [ ]:
suffixes = [f'_{model_name}' for model_name in model_names]
styles = ['r.'] + ['b-' for model_name in model_names]

legends = ['VCT'] + model_names

captive_plot(df_captive=df_captive_all, suffixes=suffixes, 
             legends = legends, styles=styles, alpha=0.2, lw=2, add_legend=False)

In [ ]:
df_results = pd.DataFrame()
result = model_vct.zigzag(u0=2, angle=30)

for model_name, model in models.items():
    result_ = model.simulate(result.result)
    df_ = result_.result
    df_['t'] = df_.index
    df_['model_name'] = model_name
    df_results = df_results.append(df_, ignore_index=True)
    

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot

fig,ax=plt.subplots()
fig.set_size_inches(10,10)

for model_name, df_ in df_results.groupby(by='model_name'):
    
    df_.plot(x='x0', y='y0', ax=ax, alpha=0.2, lw=3, style='b-')

result.result.plot(x='x0', y='y0', style='k-', zorder=10, ax=ax)
    
ax.set_xlabel('x0 [m]')   
ax.set_ylabel('y0 [m]')   

ax.set_aspect("equal")
ax.set_title("Track plot")
ax.get_legend().set_visible(False)
ax.grid(True)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[-2:],['simulations','model test'])

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(14,3)

df_results['psi_deg'] = np.rad2deg(df_results['psi'])

df_results_ = result.result.copy()
df_results_['-delta_deg'] =-np.rad2deg(df_results_['delta'])
df_results_['psi_deg'] = np.rad2deg(df_results_['psi'])

for model_name, df_ in df_results.groupby(by='model_name'):
    
    df_.plot(x='t', y='psi_deg', ax=ax, alpha=0.2, lw=3, style='b-')

df_results_.plot(y='psi_deg', ax=ax, style='k-', zorder=10)
df_results_.plot(y='-delta_deg', ax=ax, style='m-', zorder=10)
    
ax.set_xlabel('time [s]')   
ax.set_ylabel('Heading $\psi$ [deg]')   

ax.set_title("ZigZag30/30")

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[-3:],['alternative models','model','rudder angle'])

ax.grid(True)
ax.set_ylim(-60,60)

In [ ]:
model_vct.parameters